<a href="https://colab.research.google.com/github/VincentCsNv/SymbioseManagement/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('test')

test
